# KEPCO Test

## 1 Environment

In [1]:
%reload_ext autoreload
%autoreload 2

import logging
import logger
import account

from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By

from org.kepco import kepco

def log():
    return logging.getLogger(__package__)

logger.logger_init()

driver = kepco.create_driver()

import account
id=account.account_get("kepco", "id")
pw=account.account_get("kepco", "pw")
cert=account.account_get("kepco", "cert")

;

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 104.0.1293
[WDM] - Get LATEST edgedriver version for 104.0.1293 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/104.0.1293.63/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\Jaeyoung\.wdm\drivers\edgedriver\win64\104.0.1293.63]


''

## 2. Kepco Login

In [2]:
%autoreload 2
from org.kepco import *

def login(driver: WebDriver, id, pw):
    log().debug("login")
    kepco_go_homepage(driver)
    kepco_login(driver, id, pw)

login(driver, id, pw)


2022-08-22 07:36:35,419 [D] org.kepco: login


Exception: Failed to find frame. locator=('id', 'mdiiframe-1010-iframeEl')

## 2. Register

In [ ]:
%autoreload 2

from auto import *
from org.kepco import *

def register(driver: WebDriver, number, *, cert=None):

    # close_all_tab
    close_all_tab(driver)

    # # 1. 공고번호 조회
    if not open_register_tab(driver):
        log().error("Failed to open register tab.")
        
    # 1.2 조회
    log().info("1.2 search")
    # ID: Kepco - 입찰 - 공고번호 검색
    # 공고번호 입력
    num_input = wait_clickable(driver, (By.XPATH, '//input[@title="공고번호"]'))
    num_input.send_keys(number)

    # 조회 버튼 클릭                                    
    search_btn = wait_clickable(driver, (By.XPATH, '//span[text()="조회"]'))
    search_btn.click()

    #  1.3 
    log().info("1.3 validate")
    if kepco_pre_is_current_registered(driver):
        log().info("Already registered")
        return True

    log().info("1.4 apply")
    if not kepco_pre_apply_participation(driver):
        log().error("Failed to apply")
        return False

    # 2. Application Form
    # 2.1.  메세지등의 팝업을 처리한다.
    log().info("2.1 close popup")
    kepco_pre_close_popup(driver)

    # 2.2. 중소기업확인서 첨부
    log().info("2.2 attach a file")
    filepath = os.path.join(os.path.expanduser("~"), ".iaa", "AR_중소기업_확인서.pdf")
    kepco_attach_small_business_confirmation_document(driver, filepath)

    # XXX: 빠르게 진행하기 때문에 다시 제출하라는 문구가 뜨는 것 같음
    time.sleep(10)

    # 모든 조항에 동의
    log().info("2.3 aggreement")
    kepco_pre_agree_commitments(driver)

    # XXX: 빠르게 진행하기 때문에 다시 제출하라는 문구가 뜨는 것 같음
    time.sleep(10)

    # 2.3 제출 버튼 클릭
    log().info("2.4 submit")
    kepco_pre_submit_application_form(driver)

    # 3. 정리
    # 3.1 확인
    # XXX: 제출 버튼을 클릭한 후, popup이open될 때 까지 기다린다.
    log().info("3.1 confirm")
    if not kepco_pre_confirm_submission(driver):
        # "입찰참가신청등록 화면에 있는 닫기버튼을 눌러 다시 신청하세요."
        return False

    # 3.2 certificate login (Optional)
    if cert:
        log().info("3.2 certifiate")
        kepco_certificate_login(driver, cert)

    # 확인 버튼 클릭(알림: 제출 하였습니다.
    log().info("3.3 done")
    kepco_pre_confirm_done(driver)

    return True





## 3. Participate

In [ ]:

%autoreload 2

import pyautogui
from org.kepco import *
import random
from selenium.webdriver.support import expected_conditions as when
from selenium.webdriver.support.ui import WebDriverWait



#--------------------------

# ID: Kepco - 입찰 - 검색결과 검증
# Description: 투찰진행상태를 확인한다. - 미제출
def can_participate(driver:WebDriver) -> bool :
    elem = auto_find_element(driver, By.XPATH, "//div[2]/div/div[2]/table/tbody/tr/td[2]/div")
    if elem.text == "미제출":
        return True
    else:
        return False


# ID: Kepco - 입찰 - 검색결과 검증
def validate_bid_search_result(driver:WebDriver, number):
    es = auto_find_all_elements(driver, By.XPATH, '//div[@class="x-grid-row-checker"]')
    if len(es) != 1:
        log().error(f"Failed to validate the result. Item should be only one. found={len(es)} ")
        return False

    # validate number
    e = auto_find_element(driver, By.XPATH, '//div[2]/table[1]/tbody/tr/td[5]/div[contains(@class,"x-grid-cell-inner")]')
    bid_num = e.text
    if number.find(bid_num) < 0:
        log().error(f"Failed to validate the result. Bid number missmatch. found={bid_num}, expected={number} ")
        return False

    return True
        



def participate(driver, number, cost):
    # close all tabs
    close_all_tab(driver)

    # open bid tab
    if not open_bid_tab(driver):
        log().error("Failed to open bind tab.")
        return False

    # ID: Kepco - 입찰 - 공고번호 검색
    # 공고번호 입력
    num_input = wait_clickable(driver, (By.XPATH, '//input[@title="공고번호"]'))
    num_input.send_keys(number)

    # 조회 버튼 클릭                                    
    # XXX: 가끔 click이 안되는 경우가 있다. 
    # auto_click(driver, By.XPATH, '//span[text()="조회"]')
    search_btn = wait_clickable(driver, (By.XPATH, '//span[text()="조회"]'))
    search_btn.click()
    
    # messagebox : 공고일자의 최대 검색일자는 6개월 입니다. "확인" 
    msgbox = kepco_get_messagebox(driver)    
    if msgbox:
        log().info(f"messagebox: text={msgbox.text}")
        okbtn = kepco_messagebox_get_button(msgbox, "확인")
        okbtn.click()

    # ID: Kepco - 입찰 - 검색결과 검증
    if not validate_bid_search_result(driver, number):
        log().error("Validation Failed.")
        return False

    # Description: 투찰진행상태 가져오기 
    if not can_participate(driver):
        log().info("Already Registered.")
        return True

    # ID: Kepco - 입찰 - 입찰참여 버튼
    # checkbox click
    log().info(f"Kepco - 입찰 - 입찰참여 버튼")
    auto_click(driver, By.XPATH, '//div[@class="x-grid-row-checker"]')
    # TODO: 안눌리는 경우가 생긴다.
    if not auto_click(driver, By.XPATH, '//span[text()="입찰참여"]'):
        log().error("Failed to click 입찰참여 button.")
        return False
                
    # messagebox - 입찰 창여 하시겠습니까?
    msgbox = kepco_get_messagebox(driver)
    log().info(f"messagebox: text={msgbox.text}")
    yesbtn = kepco_messagebox_get_button(msgbox, "예")
    yesbtn.click()

    # ID: Kepco - 입찰 - 입찰서 작성 - 지문인식 투찰 버튼 
    log().info(f"Kepco - 입찰 - 입찰서 작성 - 지문인식 투찰 버튼")
    auto_click(driver, By.XPATH, '//td[5]/div/img')


    # TODO: 시간이 오래된 경우 공인인증서 확인 창이 생성된다. 


    # messagebox - 지문인식투찰을 진행하시겠습니까?
    msgbox = kepco_get_messagebox(driver)
    log().info(f"messagebox: text={msgbox.text}")
    okbtn = kepco_messagebox_get_button(msgbox, "예")
    okbtn.click()

    # close all popup 
    log().info(f"Kepco - Common - 모든 팝업창 닫기")
    close_all_popup(driver, timeout = 15)


    # ID: Kepco - 입찰 - 추첨
    log().info(f"Kepco - Common - 추첨번호 선택")
    boxes = auto_find_all_elements(driver, By.XPATH, "//tr[3]/td/div/div/div/table/tbody/tr/td/a/span/span/span[2]")
    boxes = random.sample(boxes, k=4)
    for box in boxes:
        # 4개가 모두 click이 안되는 경우가 있다. 
        # 중복된 개체가 있는 것으로 보인다. 
        time.sleep(1)
        auto_click(driver, box)

    # ID: Kepco - 입찰 - 가격입력
    # ---------------------------
    cost_box = wait_element(driver, (By.XPATH, "//tbody/tr[4]/td/div[1]/div/div/table/tbody/tr[1]/td/div[1]"))

    # input layer
    input_layer = cost_box.find_element(By.XPATH, './/input')
    driver.execute_script(f'arguments[0].value = "{cost}"', input_layer)

    # focus
    text_layer = cost_box.find_element(By.XPATH, './/div[@class="x-form-field-inputcover-displayEl"]')
    text_layer.click()

    valid_box = wait_element(driver, (By.XPATH, "//tbody/tr[4]/td/div[1]/div/div/table/tbody/tr[2]/td/div[1]"))

    # input layer
    input_layer = valid_box.find_element(By.XPATH, './/input')
    driver.execute_script(f'arguments[0].value = "{cost}"', input_layer)

    # focus
    text_layer = valid_box.find_element(By.XPATH, './/div[@class="x-form-field-inputcover-displayEl"]')
    text_layer.click()

    # focus out to temp element.
    cost_box.click()

    # 입력값 확인 버튼 
    auto_click(driver, By.XPATH, '//span[text()="입력값확인"]')
    
    # ---------------------------
    # ID: Kepco - 입찰 - 제출
    # ---------------------------
    auto_click(driver, By.XPATH, '//span[text()="제출"]')
    msgbox = kepco_get_messagebox(driver)
    yesbtn = kepco_messagebox_get_button(msgbox, "예")
    yesbtn.click()

    #message box - 제출되었습니다. 
    msgbox = kepco_get_messagebox(driver)
    btn = kepco_messagebox_get_button(msgbox, "확인")
    btn.click()




In [ ]:

# number= 'G092200035-00'
# register(driver, number)

number = 'G052200237-00'
cost = "25436532"
participate(driver, number, cost)


2022-06-27 23:54:15,206 [E] auto: Failed to find an element. locator=('xpath', '//span[@class="x-tab-close-btn"]'), timeout=10
2022-06-27 23:54:27,297 [E] org.kepco: Message box is none
2022-06-27 23:54:27,813 [E] org.kepco: Message box is none
2022-06-27 23:54:28,317 [E] org.kepco: Message box is none
2022-06-27 23:54:28,830 [E] org.kepco: Message box is none
2022-06-27 23:54:29,335 [E] org.kepco: Message box is none
2022-06-27 23:54:29,839 [E] org.kepco: Message box is none
2022-06-27 23:54:30,354 [E] org.kepco: Message box is none
2022-06-27 23:54:30,858 [E] org.kepco: Message box is none
2022-06-27 23:54:31,364 [E] org.kepco: Message box is none
2022-06-27 23:54:31,879 [E] org.kepco: Message box is none
2022-06-27 23:54:32,392 [E] org.kepco: Message box is none
2022-06-27 23:54:32,895 [E] org.kepco: Message box is none
2022-06-27 23:54:33,409 [E] org.kepco: Message box is none
2022-06-27 23:54:33,925 [E] org.kepco: Message box is none
2022-06-27 23:54:34,435 [E] org.kepco: Message 

False

# Kepco Close All Tab test


In [2]:
from org.kepco import Kepco
import account 

id = account.account_get("kepco", "id")
pw = account.account_get("kepco", "pw")

kepco = Kepco(id, pw)



[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 104.0.1293
[WDM] - Get LATEST edgedriver version for 104.0.1293 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/104.0.1293.63/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\Jaeyoung\.wdm\drivers\edgedriver\win64\104.0.1293.63]


In [ ]:

# Refresh
kepco.driver.refresh()

# Accept alert
alert = kepco.driver.switch_to.alert
print(alert.text)
alert.accept()



In [ ]:
from auto import *

# validate login
# wait_element(kepco.driver, (By.XPATH, '//span[text()="로그아웃"]'), timeout=15)
wait_element(kepco.driver, (By.XPATH, '//span[text()="로그인"]/../../../a[contains(@style, display: none;)]'), timeout=15)

//a[contains(@style, "display: none;")]/span/span/span[2 and text()="로그인"]



<selenium.webdriver.remote.webelement.WebElement (session="e07bd50ee9ebb7baee24d4cbc3cef519", element="db24fd16-1bce-4877-8ddf-0b3267a5144a")>